# Data Dashboard
This page is about data dashbooard to collect useful insight

In [49]:
import os
import glob
import psycopg2
import pandas as pd
from datetime import datetime
from sql_queries import *

conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

## How much male / female is Listening to Sparkify


lets try to findout

In [62]:
get_gender_query = "SELECT gender, COUNT(gender) from users GROUP BY users.gender"
cur.execute(get_gender_query)
user_genders = cur.fetchall()
df = pd.DataFrame(user_genders, columns=["Gender", "Count"])
df["Gender"] = df["Gender"].map({ "M": "Male", "F": "Female" })
df["Count"] = df["Count"].astype(str) + " Users"
df

,Gender,Count
0,Male,1933 Users
1,Female,4887 Users


## The distribution of user level

In [61]:
get_level_query = "SELECT level, Count(level) from users GROUP BY users.level"
cur.execute(get_level_query)
user_levels = cur.fetchall()
df = pd.DataFrame(user_levels, columns=["Level", "Count"])
df["Count"] = df["Count"].astype(str) + " Users"
df

,Level,Count
0,free,1229 Users
1,paid,5591 Users


## TOP 10 ARTIST Listened

In [57]:
most_listened_artist_query = """
SELECT songplays.artist_id, artists.name, COUNT(songplays.artist_id) as played_count
FROM songplays JOIN artists on songplays.artist_id = artists.artist_id
GROUP BY songplays.artist_id, artists.name
ORDER BY played_count DESC
LIMIT 10
"""
cur.execute(most_listened_artist_query)
most_listened_artist = cur.fetchall()
df = pd.DataFrame(most_listened_artist, columns=["Artist Id", "Artist Name", "Num Played"])
df

,Artist Id,Artist Name,Num Played
0,ARPFHN61187FB575F6,Lupe Fiasco,5
1,ARBEBBY1187B9B43DB,Tom Petty,4
2,ARIK43K1187B9AE54C,Lionel Richie,4
3,AR558FS1187FB45658,40 Grit,3
4,ARQGYP71187FB44566,Jimmy Wakely,1
5,ARJIE2Y1187B994AB7,Line Renaud,1
6,ARXR32B1187FB57099,Gob,1
7,ARVBRGZ1187FB4675A,Gwen Stefani,1
8,ARKULSX1187FB45F84,Trafik,1
9,ARD842G1187B997376,Blue Rodeo,1


## TOP 10 songs listened

In [56]:
most_listened_song_query = """
SELECT songplays.song_id, artists.name, songs.title,  COUNT(songplays.song_id) as played_count
FROM songplays 
JOIN songs on songplays.song_id = songs.song_id
JOIN artists on songplays.artist_id = artists.artist_id
GROUP BY songplays.song_id, songs.title, artists.name
ORDER BY played_count DESC
LIMIT 10
"""
cur.execute(most_listened_song_query)
most_listened_song = cur.fetchall()
df = pd.DataFrame(most_listened_song, columns=["Song Id", "Artist Name", "Song Title", "Num Played"])
df

,Song Id,Artist Name,Song Title,Num Played
0,SOWQTQZ12A58A7B63E,Lupe Fiasco,Streets On Fire (Explicit Album Version),5
1,SOBONFF12A6D4F84D8,Lionel Richie,Tonight Will Be Alright,4
2,SOFFKZS12AB017F194,Tom Petty,A Higher Place (Album Version),4
3,SOGDBUF12A8C140FAA,40 Grit,Intro,3
4,SOFSOCN12A8C143F5D,Gob,Face the Ashes,1
5,SONWXQJ12A8C134D94,Sophie B. Hawkins,The Ballad Of Sleeping Beauty,1
6,SOQVMXR12A81C21483,Trafik,Salt In NYC,1
7,SOZCTXZ12AB0182364,Elena,Setanta matins,1
8,SOUPIRU12A6D4FA1E1,Line Renaud,Der Kleine Dompfaff,1
9,SOWTBJW12AC468AC6E,Jimmy Wakely,Broken-Down Merry-Go-Round,1


## Type of user agent user used to listened
can be used to determine if user use desktop / mobile

In [60]:
most_user_agent_query = """
SELECT songplays.user_agent, COUNT(songplays.user_agent) as total FROM songplays GROUP BY  songplays.user_agent
ORDER BY total DESC
LIMIT 20
"""
cur.execute(most_user_agent_query)
most_user_agent = cur.fetchall()
df = pd.DataFrame(most_user_agent, columns=["User Agent", "Num of Users"])
df

,User Agent,Num of Users
0,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",971
1,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",708
2,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,696
3,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",577
4,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",573
5,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,443
6,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",427
7,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",419
8,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",319
9,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,310


## Num Of Active User
How much user is actually listened to a song ??

In [54]:
active_user_query = """
SELECT COUNT(*)
FROM songplays RIGHT JOIN users on songplays.user_id = users.user_id
GROUP BY songplays.user_id
"""
cur.execute(active_user_query)
active_user = cur.fetchall()
df = pd.DataFrame([len(active_user)], columns=["Total Active user"])
df

,Total Active user
0,96


## The most active user

In [65]:
most_active_user_query = """
SELECT songplays.user_id, users.first_name, users.last_name, COUNT(songplays.user_id) as total
FROM songplays RIGHT JOIN users on songplays.user_id = users.user_id
GROUP BY songplays.user_id, users.first_name, users.last_name
ORDER BY total DESC
LIMIT 10
"""
cur.execute(most_active_user_query)
most_active_user = cur.fetchall()
df = pd.DataFrame(most_active_user, columns=["User ID", "First Name", "Last Name", "Total SongPlayed"])
df

,User ID,First Name,Last Name,Total SongPlayed
0,49,Chloe,Cuevas,474721
1,80,Tegan,Levine,442225
2,97,Kate,Harrell,310249
3,15,Lily,Koch,214369
4,44,Aleena,Kirby,157609
5,29,Jacqueline,Lynch,119716
6,24,Layla,Griffin,103041
7,73,Jacob,Klein,83521
8,88,Mohammad,Rodriguez,72900
9,36,Matthew,Jones,61504
